In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from gensim.models import KeyedVectors
import pandas as pd
import emblaze
from emblaze.utils import Field, ProjectionTechnique
from collections import defaultdict, namedtuple

In [18]:
# fname = '../data/embedding_ggvec_full_posonly_wextra_top40_10d_order2.w2v'
# fname = '../data/pymde_10d_100n.w2v'
fname = '../data/pymde_3d_40n.w2v'
wv = KeyedVectors.load_word2vec_format(fname)

FileNotFoundError: [Errno 2] No such file or directory: '../data/pymde_3d_40n.w2v'

In [19]:
all_animes_pd = pd.read_csv('../data/processed-metadata.csv')

all_animes = []

for anime in all_animes_pd.itertuples(index=False):
    all_animes.append(anime)

print(f"anime count: {len(all_animes)}; wv len: {len(wv.index_to_key)}")

anime count: 13647; wv len: 13647


In [20]:
def ix_to_ix(ix):
    return int(wv.index_to_key[ix])

def anime_by_ix(ix):
    ix = ix_to_ix(ix)
    return all_animes[ix]

positions = wv.vectors
names = [anime_by_ix(index).title for index in range(len(wv.vectors))]
colors = [anime_by_ix(index).aired_from_year if int(anime_by_ix(index).aired_from_year) > 2000 else 2000 for index in range(len(wv.vectors))]
sizes = [math.sqrt(anime_by_ix(index).rating_count) / 110. + 1. for index in range(len(wv.vectors))]

emb = emblaze.Embedding({Field.POSITION: positions, Field.NAME: names, Field.COLOR: colors, Field.RADIUS: sizes}, n_neighbors=20,)
emb.compute_neighbors(metric='cosine')

In [21]:
# variants = emblaze.EmbeddingSet([
#     emb.project(method=ProjectionTechnique.TSNE, perplexity=33, learning_rate='auto', init='pca', early_exaggeration=1.2),
#     emb.project(method=ProjectionTechnique.TSNE, perplexity=33, learning_rate='auto', init='pca', early_exaggeration=1.3),
#     emb.project(method=ProjectionTechnique.TSNE, perplexity=33, learning_rate='auto', init='pca', early_exaggeration=1.4),
#     emb.project(method=ProjectionTechnique.TSNE, perplexity=33, learning_rate='auto', init='pca', early_exaggeration=1.5),
# ])
variants = emblaze.EmbeddingSet([
    # emb.project(method=ProjectionTechnique.TSNE, perplexity=22, learning_rate='auto', init='pca', early_exaggeration=1.3),
    # emb.project(method=ProjectionTechnique.TSNE, perplexity=25, learning_rate='auto', init='pca', early_exaggeration=1.3),
    # emb.project(method=ProjectionTechnique.TSNE, perplexity=30, learning_rate='auto', init='pca', early_exaggeration=1.3),
    # emb.project(method=ProjectionTechnique.TSNE, perplexity=35, learning_rate='auto', init='pca', early_exaggeration=1.3),
    emb.project(method=ProjectionTechnique.TSNE, perplexity=40, learning_rate='auto', init='pca', early_exaggeration=1.3),
])
thumbnails = emblaze.TextThumbnails(names)
w = emblaze.Viewer(embeddings=variants, thumbnails=thumbnails)
w

Viewer(colorScheme='plasma', data={'data': [{'_format': 'compressed', '_idtype': 'u2', '_length': 13647, 'ids'…

In [22]:
import json
import math

projection = emb.project(method=ProjectionTechnique.TSNE, perplexity=40, learning_rate='auto', init='pca', early_exaggeration=1.3)

serialized = projection.to_json(compressed=False, save_neighbors=False)
serialized['ids'] = [all_animes[ix_to_ix(i)].id for i in range(len(wv.index_to_key))]
neighbors_json = emb.get_neighbors().to_json(compressed=False)
neighbors_json['neighbors'] = {int(key):value for (key,value) in neighbors_json['neighbors'].items()}
serialized['neighbors'] = neighbors_json
with open('../data/projected_embedding_pymde_3d_40n.json', 'wt') as f:
    f.write(json.dumps(serialized))